In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import class_weight
from PIL import Image, ImageDraw, ImageFont
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import Xception
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
sys.path.append('/content/drive/My Drive/Data/')
from analysis_util import *

In [4]:
with open('/content/drive/My Drive/Data/padded/X_padded_80.pkl', 'rb') as fp:
  X = pickle.load(fp)

with open('/content/drive/My Drive/Data/resize/y.pkl', 'rb') as fp:
  y_classes = pickle.load(fp)

In [5]:
y = np.array(pd.get_dummies(y_classes))

In [6]:
# classes = get_classes(y_train)
# cw = dict(enumerate(class_weight.compute_class_weight('balanced', np.unique(classes), classes)))

In [7]:
val_test_size = int(len(y)*.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=val_test_size, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_test_size, random_state=42)

In [8]:
print(len(X_train))
print(len(X_val))
print(len(X_test))

478426
102519
102519


In [9]:
print(len(y_train))
print(len(y_val))
print(len(y_test))

478426
102519
102519


In [10]:
dropout_rate=.3

input = keras.layers.Input(shape=(80, 80, 1))

# A convolution block
x = keras.layers.Conv2D(16, kernel_size=3, strides=1, padding='same')(input)
x = keras.layers.Dropout(dropout_rate)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)

x = keras.layers.Conv2D(32, kernel_size=3, strides=1, padding='same')(x)
x = keras.layers.Dropout(dropout_rate)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)

x = keras.layers.Conv2D(64, kernel_size=3, strides=1, padding='same')(x)
x = keras.layers.Dropout(dropout_rate)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)

x = keras.layers.Conv2D(128, kernel_size=3, strides=1, padding='same')(x)
x = keras.layers.Dropout(dropout_rate)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)


# Uncomment the below line if you're using dense layers
# x = tf.keras.layers.GlobalMaxPooling2D()(x)

# Fully connected layer 1
# x = tf.keras.layers.Dropout(dropout_rate)(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Dense(units=64)(x)
# x = tf.keras.layers.Activation('relu')(x)

# Fully connected layer 1
x = keras.layers.Conv2D(1028, kernel_size=1, strides=1)(x)
x = keras.layers.Dropout(dropout_rate)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)

# Fully connected layer 2
# x = tf.keras.layers.Dropout(dropout_rate)(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Dense(units=len_classes)(x)
# predictions = tf.keras.layers.Activation('softmax')(x)

# Fully connected layer 2
x = keras.layers.Conv2D(514, kernel_size=1, strides=1)(x)
x = keras.layers.Dropout(dropout_rate)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.GlobalMaxPooling2D()(x)
predictions = keras.layers.Activation('softmax')(x)

model = keras.Model(inputs=input, outputs=predictions)
print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 80, 80, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 80, 80, 16)        160       
_________________________________________________________________
dropout (Dropout)            (None, 80, 80, 16)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 80, 80, 16)        64        
_________________________________________________________________
activation (Activation)      (None, 80, 80, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 80, 80, 32)        4640      
_________________________________________________________________
dropout_1 (Dropout)          (None, 80, 80, 32)       

In [11]:
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=1e-3), metrics=['acc'])

In [12]:
# X_trial=np.asarray(X_train[0]).reshape(149,118,1)
# X_trial.shape

In [13]:
# xdim, ydim = X_train[0].size
# model.train_on_batch([X_trial], y_train[0])

In [14]:
history = model.fit(
    np.asarray([np.asarray(x) for x in X_train]).reshape(-1,80,80,1), 
    np.array(y_train), 
    epochs=1, 
    batch_size=50,
    # class_weight=cw,
    validation_data=(np.asarray([np.asarray(x) for x in X_val]).reshape(-1,80,80,1), y_val))

5192/9569 [===============>..............] - ETA: 16:50 - loss: 4.7225 - acc: 0.1129

KeyboardInterrupt: ignored

In [ ]:
visualize_training_results(history)

In [ ]:
yhat_test = get_classes(model.predict(np.array(X_test).reshape(-1,80,80,1)))
dummy_to_unicode = dict(enumerate(pd.get_dummies(y_classes).columns))

In [ ]:
print(classification_report(get_classes(y_test), yhat_test))

In [ ]:
misclassifications = misclass_report(get_classes(y_test), yhat_test, dummy_to_unicode)

In [ ]:
index=45
print(chr(int(dummy_to_unicode[index][2:], 16)))
# print(513)
misclassifications[index]

In [ ]:
# model.save('/content/drive/My Drive/Models/fcn')